# Inhaltsverzeichnis des Skripts:

[[_TOC_]]

---

### Vorliegende Datentyp:

**quantitative Daten auf Kardinalskala**

*Vorliegende Columns:*

- **Date:** Datum
- **Open:** Eröffnungskurs
- **High:** Höchstkurs
- **Low:** Tiefkurs
- **Close:** Schlusskurs
- **Volume:** Handelsvolumen

In [1]:
########################################################
#--------------- benötigten Librarys ------------------#
########################################################

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/kaihufnagel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Download und Speicherung der Datei

In [ ]:
def download_stock_data(ticker_symbol, time_period, csv_file_location):
    """
    Lädt Aktiendaten von Yahoo Finance herunter und speichert sie als CSV.
    
    Args:
        ticker_symbol (str): Das Ticker-Symbol der Aktie (z.B. "MSFT" für Microsoft)
        time_period (str): Der Zeitraum für die Daten (z.B. "5y", "1mo", "max")
        csv_file_location (str): Der Dateipfad der csv-Datei. Diese Datei sollte im Repo-Ordner 'data' liegen
    
    Returns:
        pandas.DataFrame: Der heruntergeladene Datensatz
    """
    
    # Ticker-Objekt abrufen
    stock = yf.Ticker(ticker_symbol)
    
    # Historische Daten herunterladen
    hist_data = stock.history(period=time_period)
    
    # Daten als CSV speichern
    hist_data.to_csv(csv_file_location)
    print(f"Daten wurden in {csv_file_location} gespeichert.")

    # Konvertiere alle numerischen Spalten zu Float
    for col in hist_data.columns:
        hist_data[col] = pd.to_numeric(hist_data[col], errors='coerce')

    # Allgemeine Infos
    print(f"+{'-'*48}+")
    print(f"|{'Allgemeine Informationen der Aktien-Daten':^48}|")
    print(f"+{'-'*48}+ \n")
    hist_data.info()
    
    # Statistische Zusammenfassung
    data_zusammenfassung = hist_data.describe()
    print(f"\n\n")
    print(f"+{'-'*48}+")
    print(f"|{'Statistische Zusammenfassung der Aktien-Daten':^48}|")
    print(f"+{'-'*48}+ \n")
    print(data_zusammenfassung)

    return hist_data, data_zusammenfassung


# Verwendung der Funktion
if __name__ == "__main__":
    # Microsoft Daten für 5 Jahre herunterladen
    msft_data = download_stock_data("MSFT", "max", "/Users/kaihufnagel/Documents/BBA/6. Semester/SP BBA/Menden/THWS_Reihenanalysen/data/microsoft_aktien_daten.csv", 1)

Daten wurden in /Users/kaihufnagel/Documents/BBA/6. Semester/SP BBA/Menden/THWS_Reihenanalysen/data/microsoft_aktien_daten.csv gespeichert.
+------------------------------------------------+
|   Allgemeine Informationen der Aktien-Daten    |
+------------------------------------------------+ 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9867 entries, 1986-03-13 00:00:00-05:00 to 2025-05-09 00:00:00-04:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          9867 non-null   float64
 1   High          9867 non-null   float64
 2   Low           9867 non-null   float64
 3   Close         9867 non-null   float64
 4   Volume        9867 non-null   int64  
 5   Dividends     9867 non-null   float64
 6   Stock Splits  9867 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 616.7 KB



+------------------------------------------------+
| Statistische Zusammenfassung der Aktien-Daten  |
+------